<a href="https://colab.research.google.com/github/FranciscoRamirezArias/Pruebas_Software_Aseguramiento_Calidad/blob/main/A01316379_A5.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instituto Tecnologico de Monterrey**
# **Maestría en Inteligencia Artificial Aplicada**
# **Curso: Pruebas de Software y aseguramiento de la Calidad**
# **Clave: TC4017.10**
## Profesor Títular: Dr. Gerardo Padilla Zárate
## Profesora Asistente: Mtra. Viridiana Rodríguez González
## Estudiante: Francisco Javier Ramírez Arias
## Matrícula: A01316379

##Actividad: Actividad 5.2
##Ejercicio de Programación #2
##Descripción: Cálculo de ventas.

# Instalación de Pylint

In [1]:
pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.4/536.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0:00:00


# Instalación de Flake

In [2]:
pip install flake8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.5 MB/s eta 0:00:00


# Código de Cálculo de Ventas

In [9]:
# Nos aseguramos de utilizar Pythno Versión 3

#!/usr/bin/env python3

%%writefile computeSales.py
"""A module for computing sales from price catalogues and sales records."""

import sys       # Capturar argumentos de la línea de comandos
import json      # Leer los archivos JSON
import time      # Medir tiempo de ejecución del script


# DEFICIÓN DE FUNCIÓN
def load_json_file(file_path):
    """
    Carga el contenido del un archivo JSON y lo devuelve como un objeto Python.

    Argumentos:
        file_path (str): La ruta del archivo a leer.

    Regresa:
        dict/list: Los datos cargados si el proceso es exitoso.
         None: Si ocurre error de lectura o formato incorrecto.
    """
    try:
        # Abrimos el archivo con UTF-8 (Compatibilidad).
        with open(file_path, "r", encoding="utf-8") as file:
            return json.load(file)
    except FileNotFoundError:
        # Manejo específico: El archivo no existe en la ruta dada.
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError as error:
        # Manejo específico: El archivo existe pero no valido (JSON)
        print(f"Error: Invalid JSON format in '{file_path}': {error}")
        return None


# DEFICIÓN DE FUNCIÓN
def build_price_dictionary(price_catalogue):
    """
    Construye un diccionario de precios a partir de un catálogo de productos.

    Argumentos:
        price_catalogue (list): Una lista de diccionarios que representan
                                productos.

    Regresa:
        dict: Un diccionario donde las claves son los nombres de los productos
              y los valores son los precios correspondientes.
    """
    price_dict = {}

    for item in price_catalogue:
        try:
            # Extracción de los datos
            title = item["title"]            # Extracción Nombre
            price = float(item["price"])     # Extracción Precio
            price_dict[title] = price
        except (KeyError, ValueError, TypeError):
            # Captura: Tipos de datos incorrectos
            print(f"Warning: Invalid product entry ignored: {item}")

    return price_dict


# DEFICIÓN DE FUNCIÓN
def compute_total_sales(price_dict, sales_record):
    """
    Calcula las ventas totales y reporte detallado.

    Busca en los registros de ventas con el diccionario de precios.
    Realiza validaciones de existencia del producto y consistencia
    en las cantidades.

    Argumentos:
        price_dict (dict): Catálogo de precios {Nombre: Precio}.
        sales_record (list): Transacciones de venta.

    Regresa:
        tuple: que contiene:
            - total_cost (float): Suma acumulada de las ventas.
            - detailed_lines (list): Lista formateado con el detalle
              de cada producto, cantidad, precio unitario y subtotal.

    Nota:
        La función ignora entradas con productos inexistentes en el catálogo
    """

    # Inicialización de variables
    total_cost = 0.0
    detailed_lines = []

    for entry in sales_record:
        try:
            # Extracción de los datos de la venta
            product = entry["Product"]
            quantity = int(entry["Quantity"])
            # El poducto debe de existir en el catálogo de precios
            if product not in price_dict:
                print(f"Warning: Product '{product}' not found in catalogue.")
                continue
            # Calculo de las ventas
            unit_price = price_dict[product]
            subtotal = unit_price * quantity
            total_cost += subtotal
            # Formateo de las líneas para el reporte
            detailed_lines.append(
                f"Product: {product:<30} "
                f"Qty: {quantity:<5} "
                f"Unit Price: ${unit_price:<8.2f} "
                f"Subtotal: ${subtotal:.2f}"
            )

        except (KeyError, ValueError, TypeError):
            # Manejo de errores si faltan campos
            print(f"Warning: Invalid sale entry ignored: {entry}")

    return total_cost, detailed_lines


# DEFICIÓN DE FUNCIÓN
def write_results(report_lines):
    """
    Escribe un reporte de resultados en un archivo de texto.

    Argumentos:
        report_lines: lista de strings, donde cada elemento representa una
        línea del reporte

    Regresa:
        None: La función escribe a disco
    """
    # Abrimos en archivo en modo escrotura para generar el reporte
    with open("SalesResults.txt", "w", encoding="utf-8") as file:
        for line in report_lines:
            # Escribimos cada línea y damos un salto de línea
            file.write(line + "\n")


# PROGRAMA PRINCIPAL
def main():
    """
    Punto de entrada principal del script.
        Flujo completo de ejecución:
        1. Inicia el cronómetro de alta precisión.
        2. Valida los argumentos de la línea de comandos.
        3. Carga los archivos JSON de precios y ventas.
        4. Procesa los datos para calcular el total.
        5. Genera un reporte formateado tanto en consola
           como en un archivo externo.
    """
    # Iniciamos el timer
    start_time = time.perf_counter()

    # Validación de argumentos el script espera 2 archivo
    if len(sys.argv) != 3:
        print(
            "Usage: python computeSales.py "
            "priceCatalogue.json salesRecord.json"
        )
        sys.exit(1)

    # Asignación de las rutas desde la terminal
    price_file = sys.argv[1]
    sales_file = sys.argv[2]

    # Se cargan los datos
    price_catalogue = load_json_file(price_file)
    sales_record = load_json_file(sales_file)
    # Si alguno de los archivos falló salimos del programa
    if price_catalogue is None or sales_record is None:
        sys.exit(1)

    # Paso 1: Convertir catálogo a diccionario
    price_dict = build_price_dictionary(price_catalogue)

    # Paso 2: Cálculo de ventas y líneas
    total_cost, detailed_lines = compute_total_sales(
        price_dict,
        sales_record,
    )
    # Finalizamos timer
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    # Estructura del reporte
    report = []
    report.append("==== COMPANY SALES REPORT ====")
    report.append("")
    report.extend(detailed_lines)
    report.append("")
    report.append(f"TOTAL COMPANY SALES: ${total_cost:.2f}")
    report.append(
        f"Execution Time (seconds): {elapsed_time:.6f}"
    )
    # Salida doble: Exritura en pantalla y archivo
    for line in report:
        print(line)

    write_results(report)


# Aseguramos que main() solo se ejecute si el script es llamado directamente
if __name__ == "__main__":
    main()
##

Overwriting computeSales.py


# Análisis de Código con Pylint y Flake

In [4]:
!pylint computeSales.py

************* Module computeSales
computeSales.py:1:0: C0103: Module name "computeSales" doesn't conform to snake_case naming style (invalid-name)

-----------------------------------
Your code has been rated at 9.86/10



In [ ]:
!flake8 computeSales.py

## **Cálculo de ventas (archivo): TC1.Sales.json**

In [10]:
!python computeSales.py TC1.ProductList.json TC1.Sales.json

==== COMPANY SALES REPORT ====

Product: Rustic breakfast               Qty: 1     Unit Price: $21.32    Subtotal: $21.32
Product: Sandwich with salad            Qty: 2     Unit Price: $22.48    Subtotal: $44.96
Product: Raw legums                     Qty: 1     Unit Price: $17.11    Subtotal: $17.11
Product: Fresh stawberry                Qty: 1     Unit Price: $28.59    Subtotal: $28.59
Product: Pears juice                    Qty: 2     Unit Price: $19.49    Subtotal: $38.98
Product: Green smoothie                 Qty: 3     Unit Price: $17.68    Subtotal: $53.04
Product: Cuban sandwiche                Qty: 2     Unit Price: $18.50    Subtotal: $37.00
Product: Hazelnut in black ceramic bowl Qty: 2     Unit Price: $27.35    Subtotal: $54.70
Product: Tomatoes                       Qty: 1     Unit Price: $26.03    Subtotal: $26.03
Product: Plums                          Qty: 1     Unit Price: $19.18    Subtotal: $19.18
Product: Fresh blueberries              Qty: 2     Unit Price: $21.0

## **Cálculo de ventas (archivo): TC2.Sales.json**

In [13]:
!python computeSales.py TC1.ProductList.json TC2.Sales.json

==== COMPANY SALES REPORT ====

Product: Rustic breakfast               Qty: 200   Unit Price: $21.32    Subtotal: $4264.00
Product: Sandwich with salad            Qty: 23    Unit Price: $22.48    Subtotal: $517.04
Product: Raw legums                     Qty: 11    Unit Price: $17.11    Subtotal: $188.21
Product: Fresh stawberry                Qty: 221   Unit Price: $28.59    Subtotal: $6318.39
Product: Raw legums                     Qty: 2     Unit Price: $17.11    Subtotal: $34.22
Product: Green smoothie                 Qty: 400   Unit Price: $17.68    Subtotal: $7072.00
Product: Cuban sandwiche                Qty: 2     Unit Price: $18.50    Subtotal: $37.00
Product: Hazelnut in black ceramic bowl Qty: 2     Unit Price: $27.35    Subtotal: $54.70
Product: Tomatoes                       Qty: 1     Unit Price: $26.03    Subtotal: $26.03
Product: Plums                          Qty: 250   Unit Price: $19.18    Subtotal: $4795.00
Product: Fresh blueberries              Qty: 334   Unit Pr

## **Cálculo de ventas (archivo): TC3.Sales.json**

In [12]:
!python computeSales.py TC1.ProductList.json TC3.Sales.json

==== COMPANY SALES REPORT ====

Product: Rustic breakfast               Qty: 200   Unit Price: $21.32    Subtotal: $4264.00
Product: Sandwich with salad            Qty: 23    Unit Price: $22.48    Subtotal: $517.04
Product: Raw legums                     Qty: 11    Unit Price: $17.11    Subtotal: $188.21
Product: Fresh stawberry                Qty: 221   Unit Price: $28.59    Subtotal: $6318.39
Product: Raw legums                     Qty: 2     Unit Price: $17.11    Subtotal: $34.22
Product: Green smoothie                 Qty: 400   Unit Price: $17.68    Subtotal: $7072.00
Product: Cuban sandwiche                Qty: 2     Unit Price: $18.50    Subtotal: $37.00
Product: Hazelnut in black ceramic bowl Qty: 2     Unit Price: $27.35    Subtotal: $54.70
Product: Tomatoes                       Qty: 1     Unit Price: $26.03    Subtotal: $26.03
Product: Plums                          Qty: 250   Unit Price: $19.18    Subtotal: $4795.00
Product: Fresh blueberries              Qty: 334   Unit Pr

#Conclusiones

El desarrollo del programa computeSales.py nos permite el cálculo total de las ventas a partir de dos fuentes de información en formato JSON: un catálogo de precios y un registro de ventas. La solucion no solamente cumple con los requisitos funcionales establecidos, también podemos observar una adecuada organización de las funciones para cargar datos, validar, procesamiento y generación de reporte.

Desde un punto de vista de algorítmica, utilizar diccionario para almacenar el catálogo nos permite tener acceso en un tiempo contante a cada producto. Lo anterior nos permite ser eficientes incluso cuando el volumen de las ventas se incrementa. La complejidad del programa es O(n), siendo n el numero de registros de ventas, lo cual es adecuado para escenarios con cientos o miles de transacciones.

La incorporacion de la medicion de tiempo de ejecución añade un componente de analisis de desempeño, permitiendo evaluar de manera objetiva la eficiencia de procesamiento. Aunado con la generación de una salida clara y legible, permitiendo que el resultado sea sencillo de entender por el usuario final.

Los resultados que se obtuvieron al correr las tres pruebas muestran lo siguiente:



1.   TC1.Sales.json
  * Total: 2481.86
  * Tiempo de Ejecución: 0.00696 segundos
2.   TC2.Sales.json
  * Total: 166568.23
  * Tiempo de Ejecución: 0.000529 segundos
3.   TC3.Sales.json
  * Total: 165235.37
  * Tiempo de Ejecución: 0.000574 segundos

Estos cálculos demuestran que el sistema está realizando correctamente:


*   Lectura del catálogo.
*   La sociación producto-precio.
*   La multiplicación por cantidad.
*   La acumulación del total general.

Las diferencias entre TC2 y TC3 sugieren que:
*  Puede haber devoluciones (cantidades negativas)
*  Puede haber variación en cantidades o productos
*  El sistema está procesando correctamente distintos escenarios de ventas.

Lo anterior nos permite obervar que al parecer no hay inconsistencias en los totales reportados.


Se procuro apegarse al estándar PEP 8 no solo para mejorar la legibilidad del código, sino para poder que este en un futuro sea sencillo de mantener y coherente al momento de realizar extensiones, incorporación de pruebas automatizadas y si desea evolucionar hacia una mayor complejidad.

Como comentario final, el programa es eficiente, se mejoro utilizando las librerias de Pylint y Flake, en el caso de Flake no se presento ningún error o sugerencia, y en el caso de Pylint muestra una recomendación, que no se siguio, esto debido para cumplir con el requisito explicito que se encuentra en los requerimientos y esta relacionado con el nombre del archivo computeSales.py.

El programa es escalable, eficiente, funcional y estable ante diferentes volúmenes de datos.